In [ ]:
from dotenv import load_dotenv
from google.adk.agents import Agent, SequentialAgent, ParallelAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

In [2]:
# load the api key
load_dotenv()

True

In [3]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504], # Retry on these HTTP errors
)

### AGENT 1 : Which only fetches news related to Hyderabad, India.

In [4]:
hyd_news_agent = Agent(
    name="hyderabad_news_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are an AI news-retrieval agent focused exclusively on Hyderabad, India.

    Your task:
    - Accept a news topic provided by the user.
    - Use the google_search tool to search ONLY for news related to that topic within the Hyderabad region.
    - Prioritize results from credible local and national news sources that report Hyderabad-specific updates.
    - Ignore news that is national, global, or related to other cities unless it directly impacts Hyderabad.
    - Summarize the findings clearly, factually, and concisely.
    - If no Hyderabad-specific results are found, state that explicitly and provide the closest relevant regional information.

    Output requirements:
    - Provide a short, well-structured summary.
    - Include only verified information from the search tool.
    - Do not add opinions or fabricated details.
    """,
    tools=[google_search],
    output_key="hyderabad_news",  
)


### AGENT 2 : Which only fetches news related to Mumbai, India.

In [5]:
mumbai_news_agent = Agent(
    name="mumbai_news_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are an AI news-retrieval agent focused exclusively on Mumbai, India.

    Your task:
    - Accept a news topic provided by the user.
    - Use the google_search tool to search ONLY for news related to that topic within the Mumbai region.
    - Prioritize results from credible local and national news sources that report Mumbai-specific updates.
    - Ignore news that is national, global, or related to other cities unless it directly impacts Mumbai.
    - Summarize the findings clearly, factually, and concisely.
    - If no Mumbai-specific results are found, state that explicitly and provide the closest relevant regional information.

    Output requirements:
    - Provide a short, well-structured summary.
    - Include only verified information from the search tool.
    - Do not add opinions or fabricated details.
    """,
    tools=[google_search],
    output_key="mumbai_news",  
)


### AGENT 3 : Which only fetches news related to Chennai, India.

In [6]:
chennai_news_agent = Agent(
    name="chennai_news_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are an AI news-retrieval agent focused exclusively on Chennai, India.

    Your task:
    - Accept a news topic provided by the user.
    - Use the google_search tool to search ONLY for news related to that topic within the Chennai region.
    - Prioritize results from credible local and national news sources that report Chennai-specific updates.
    - Ignore news that is national, global, or related to other cities unless it directly impacts Chennai.
    - Summarize the findings clearly, factually, and concisely.
    - If no Chennai-specific results are found, state that explicitly and provide the closest relevant regional information.

    Output requirements:
    - Provide a short, well-structured summary.
    - Include only verified information from the search tool.
    - Do not add opinions or fabricated details.
    """,
    tools=[google_search],
    output_key="chennai_news",  
)


### AGENT 2 : Which only fetches news related to Banglore, India.

In [7]:
banglore_news_agent = Agent(
    name="banglore_news_agent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    You are an AI news-retrieval agent focused exclusively on Banglore, India.

    Your task:
    - Accept a news topic provided by the user.
    - Use the google_search tool to search ONLY for news related to that topic within the Banglore region.
    - Prioritize results from credible local and national news sources that report Banglore-specific updates.
    - Ignore news that is national, global, or related to other cities unless it directly impacts Banglore.
    - Summarize the findings clearly, factually, and concisely.
    - If no Banglore-specific results are found, state that explicitly and provide the closest relevant regional information.

    Output requirements:
    - Provide a short, well-structured summary.
    - Include only verified information from the search tool.
    - Do not add opinions or fabricated details.
    """,
    tools=[google_search],
    output_key="banglore_news",  
)


### A Parallel Agent, which has 4 sub-agents to fetch news from different cities in India.

In [8]:
parallel_news_team = ParallelAgent(
    name="ParallelNewsFetchTeam",
    sub_agents=[hyd_news_agent, mumbai_news_agent, chennai_news_agent,banglore_news_agent],
)

### Aggregator Agent to combine all the news fetched by the parallel agent and create a summary.

In [9]:
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""
    Combine the following four regional news summaries into a single, polished Super News Outline:

    **Mumbai News:**
    {mumbai_news}

    **Hyderabad News:**
    {hyderabad_news}

    **Bangalore News:**
    {banglore_news}

    **Chennai News:**
    {chennai_news}

    Your task:
    - Clean, edit, and professionally format each city's news section.
    - Remove redundancy and ensure clarity.
    - Do not add any information not present in the inputs.
    - Highlight important developments from each city.
    - Optionally identify cross-city patterns or common themes.
    
    Final Output Format:
    1. A short executive summary (3-5 sentences).
    2. A structured breakdown:
       - **Mumbai**
       - **Hyderabad**
       - **Bangalore**
       - **Chennai**
    3. An optional “Overall Insights” section if applicable.

    Ensure the final output is concise, factual, and easy to read.
    Final output should be within 200 wrods
    """,
    output_key="executive_summary",
)


### Now putting the paralallel agent and the aggregator agent together in a workflow.

In [10]:
root_agent = SequentialAgent(
    name="NewsSystem",
    sub_agents=[parallel_news_team, aggregator_agent],
)

In [11]:
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Give me sport news"
)

App name mismatch detected. The runner is configured with app name "InMemoryRunner", but the root agent was loaded from "/Users/grshankersai/google_agentic_ai_course/venv/lib/python3.11/site-packages/google/adk/agents", which implies app name "agents".



 ### Created new session: debug_session_id

User > Give me sport news
chennai_news_agent > In Chennai, the Chennai Super Warriors have maintained a perfect record in the Indian Pickleball League, securing a 5-1 victory against the Mumbai Smashers on December 2, 2025. The team also achieved a 4-2 win over the Hyderabad Royals on the same day. Roos van Reek and Harsh Mehta were recognized as Players of the Tie in their match against the Mumbai Smashers. In other sports news for Tamil Nadu, the government is exploring the possibility of introducing IPL-style matches to revolutionize the Jallikattu festival. Additionally, the state is considering whether breakdancing could transition from an art form to an official sport in Tamil Nadu.
mumbai_news_agent > Mumbai's cricket scene is buzzing with activity. Shivam Dube, a key player in India's Asia Cup triumph, will miss the Ranji Trophy opener for Mumbai due to injury. In another Ranji Trophy match, former captain Ajinkya Rahane scored 159 r

In [12]:
print(response[-1].actions.state_delta["executive_summary"])

Here's a consolidated sports news summary:

**Executive Summary**
This week's sports news highlights include significant cricket performances in Mumbai and Bangalore, with Ajinkya Rahane and Devdutt Padikkal scoring centuries. Hyderabad celebrated victories in cricket and other sports, while Karnataka announced substantial incentives for Olympic medalists. Chennai's pickleball team remained undefeated, and the state is exploring innovative sports initiatives.

---

**1. Mumbai**
*   **Cricket:** Shivam Dube is out due to injury, while Ajinkya Rahane scored 159 runs in the Ranji Trophy. Yashasvi Jaiswal is reportedly considering a team move, and Prithvi Shaw scored a century against his former team. Sarfaraz Khan's century in the Syed Mushtaq Ali Trophy may boost his IPL prospects. The Mumbai Cricket Association announced selections for U15 Girls and U14 Boys.
*   **Other Sports:** News also covers a Manchester United football match and the Junior Hockey World Cup.

**2. Hyderabad**
*  